Mal die Daten für die 6 aktivsten Nutzer holen...

In [1]:
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True) # sample data only

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 13:41:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
dataFrame.printSchema()
dataFrame.show(1)

root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)

+--------------------+---+----+---+-----------+
|             user_id|  x|   y|  t|pixel_color|
+--------------------+---+----+---+-----------+
|vKtmwRQwAYtw5poO4...|804|1992|  0|    #FF3881|
+--------------------+---+----+---+-----------+
only showing top 1 row



In [6]:
from pyspark.sql import DataFrame,Column
import pyspark.sql.functions as F
from src.features.feature_functions import group_dataframe_by_columns


def test(df_input: DataFrame, columns_to_group: list) -> DataFrame:
    ''' groups a dataframe by a list of columns. all other columns get aggregatet as a list ov their values '''
    agg_columns = [
        colname for colname in df_input.columns if colname not in columns_to_group]
    print(agg_columns)
    exprs = [F.collect_list(colName) for colName in agg_columns]
    exprs.insert(0,F.count(columns_to_group[0]).alias('count'))
    df_grouped = df_input.groupby(columns_to_group).agg(*exprs)
    return df_grouped
testdf = test(dataFrame,["user_id"])

testdf.printSchema()
print()

['x', 'y', 't', 'pixel_color']
root
 |-- user_id: string (nullable = true)
 |-- count: long (nullable = false)
 |-- collect_list(x): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(y): array (nullable = false)
 |    |-- element: integer (containsNull = false)
 |-- collect_list(t): array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- collect_list(pixel_color): array (nullable = false)
 |    |-- element: string (containsNull = false)




In [ ]:
from src.features.feature_functions import group_dataframe_by_user
groupedFrame = group_dataframe_by_user(dataFrame)
groupedFrame.printSchema()

In [ ]:
dataCollection = groupedFrame.take(5)

In [ ]:
print(dataCollection)

In [ ]:
from src.visualization.visuialization_functions import plot_userlist_pixel

userpixel = []
for row in dataCollection:
    user_id , count ,xs , ys , ts , pixel_color = list(row)
    userpixel.append([xs,ys,ts])
print(userpixel)
plot_userlist_pixel(userpixel)


In [ ]:
from src.visualization.visuialization_functions import plot_pixellist

print(userpixel[0])
plot_pixellist(userpixel[0])